<a href="https://colab.research.google.com/github/iannstronaut/WildFire/blob/Experimental/Wildfire_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wildfire with CNN